In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Age-Related_Macular_Degeneration"
cohort = "GSE38662"

# Input paths
in_trait_dir = "../../input/GEO/Age-Related_Macular_Degeneration"
in_cohort_dir = "../../input/GEO/Age-Related_Macular_Degeneration/GSE38662"

# Output paths
out_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/GSE38662.csv"
out_gene_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/gene_data/GSE38662.csv"
out_clinical_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/clinical_data/GSE38662.csv"
json_path = "../../output/preprocess/Age-Related_Macular_Degeneration/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# This step has already been executed successfully. There's no code to debug.
# The output shows we have retrieved:
# - Background information about human embryonic stem cells
# - Sample characteristics showing cell types, cell lines, passages, and gender


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import json
import os
from typing import Optional, Callable, Dict, Any
import glob

# Let's first check what files are actually available in the cohort directory
print(f"Looking for files in: {in_cohort_dir}")
available_files = glob.glob(os.path.join(in_cohort_dir, "*"))
print("Available files:", available_files)

# Check for gene expression data (looking for common file patterns)
gene_file_patterns = ["*_series_matrix.txt", "*gene*.txt", "*expression*.txt", "*.CEL", "*.gpr"]
gene_files = []
for pattern in gene_file_patterns:
    gene_files.extend(glob.glob(os.path.join(in_cohort_dir, pattern)))

is_gene_available = len(gene_files) > 0
print(f"Gene expression files found: {gene_files}")
print(f"is_gene_available: {is_gene_available}")

# Look for clinical data files (could be in various formats)
clinical_file_patterns = ["*clinical*.txt", "*pheno*.txt", "*sample*.txt", "*_series_matrix.txt"]
clinical_files = []
for pattern in clinical_file_patterns:
    clinical_files.extend(glob.glob(os.path.join(in_cohort_dir, pattern)))

# Initialize variables
trait_row = None
age_row = None
gender_row = None
clinical_data = None

# Check if any clinical files were found
if clinical_files:
    print(f"Potential clinical data files: {clinical_files}")
    
    # Try to read the first available clinical file
    # Start with series matrix file if available as it often contains sample characteristics
    series_matrix_files = glob.glob(os.path.join(in_cohort_dir, "*_series_matrix.txt"))
    
    if series_matrix_files:
        try:
            # For series matrix files, we need to extract the sample characteristics
            with open(series_matrix_files[0], 'r') as f:
                lines = f.readlines()
            
            # Extract sample characteristic lines
            char_lines = [line for line in lines if line.startswith("!Sample_characteristics_ch")]
            
            if char_lines:
                # Convert to dataframe
                data = []
                for line in char_lines:
                    parts = line.strip().split('\t')
                    if len(parts) > 1:
                        data.append(parts[1:])  # Skip the first part which is the header
                
                if data:
                    clinical_data = pd.DataFrame(data)
                    print("Clinical data shape from series matrix:", clinical_data.shape)
                    print(clinical_data.head())
                    
                    # Print unique values for each row to identify trait, age, and gender
                    for i in range(len(clinical_data.index)):
                        unique_values = clinical_data.iloc[i].unique()
                        print(f"Row {i} unique values: {unique_values}")
                        
                        # Look for trait-related terms in the unique values
                        values_str = ' '.join(str(v).lower() for v in unique_values)
                        if any(term in values_str for term in ['amd', 'macular degeneration', 'disease', 'diagnosis', 'status']):
                            trait_row = i
                            print(f"Potential trait row found at index {i}")
                        
                        # Look for age-related terms
                        if any(term in values_str for term in ['age', 'years']):
                            age_row = i
                            print(f"Potential age row found at index {i}")
                        
                        # Look for gender-related terms
                        if any(term in values_str for term in ['gender', 'sex', 'male', 'female']):
                            gender_row = i
                            print(f"Potential gender row found at index {i}")
        except Exception as e:
            print(f"Error reading series matrix file: {e}")

# Check the background information file if it exists
background_path = os.path.join(in_cohort_dir, "background.txt")
if os.path.exists(background_path):
    with open(background_path, 'r') as f:
        background_info = f.read()
    print("\nBackground Information:")
    print(background_info)
    
    # Look for clues in background info about trait, age, and gender
    bg_lower = background_info.lower()
    
    # If we haven't found trait info yet, check background
    if trait_row is None and ('amd' in bg_lower or 'macular degeneration' in bg_lower):
        print("Background information suggests this dataset is related to AMD.")
        
    # Check for mentions of age uniformity or restrictions
    if 'all patients are' in bg_lower and 'age' in bg_lower:
        print("Background suggests age might be uniform across samples.")
        
    # Check for mentions of gender uniformity or restrictions
    if 'all patients are' in bg_lower and ('male' in bg_lower or 'female' in bg_lower):
        print("Background suggests gender might be uniform across samples.")

# Data type conversion functions
def convert_trait(value):
    """Convert trait value to binary (0: control, 1: case)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    value = str(value).lower()
    if 'control' in value or 'normal' in value or 'healthy' in value:
        return 0
    elif 'amd' in value or 'case' in value or 'patient' in value or 'disease' in value or 'macular degeneration' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        # Extract numbers if mixed with text
        import re
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return float(numbers[0])
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0: female, 1: male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    value = str(value).lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available and clinical_data is not None:
    # Use the geo_select_clinical_features function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    print("\nSelected Clinical Features Preview:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    #


### Step 3: Dataset Analysis and Clinical Feature Extraction

### Step 4: Gene Identifier Review

### Step 5: Gene Annotation

### Step 6: Gene Identifier Mapping